In [5]:
import optuna
import sqlalchemy

username  = "hogehoge"
password = "hugahuga"
dbname = ""
uri = 'postgresql+psycopg2://{}:{}@localhost:5432/{}'.format(username, password, dbname)

In [6]:
from sqlalchemy import create_engine
engine = create_engine(uri)

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.schema import Column
from sqlalchemy.types import Integer, String
from sqlalchemy.orm import sessionmaker

class User(Base):
    __tablename__ = "user"  # テーブル名を指定
    user_id = Column(Integer, primary_key=True)
    first_name = Column(String(255))
    last_name = Column(String(255))
    age = Column(Integer)

    def full_name(self):  # フルネームを返すメソッド
        return "{self.first_name} {self.last_name}"
Base.metadata.create_all(engine)
SessionClass = sessionmaker(engine)  # セッションを作るクラスを作成
session = SessionClass()
user_a = User(first_name="first_a", last_name="last_a", age=79)
user_b = User(first_name="first_a", last_name="last_a", age=79)
session.add(user_a)
session.add(user_b)
session.commit()
users = session.query(User).all()  # userテーブルの全レコードをクラスが入った配列で返す
user = session.query(User).first()  # userテーブルの最初のレコードをクラスで返す
for u in users:
    print(u.age)

In [7]:
import optuna

def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    score = (x - 2) ** 2
    #print('x: %1.3f, score: %1.3f' % (x, score))
    return score

storage = optuna.storages.RDBStorage(
    url=uri,
)
study = optuna.create_study(study_name="testes",storage=storage, load_if_exists=True)
study.optimize(objective, n_trials=5)

[I 2021-11-19 16:57:38,554] A new study created in RDB with name: testes
[I 2021-11-19 16:57:48,149] Trial 0 finished with value: 46.66515400484355 and parameters: {'x': 8.831189794233765}. Best is trial 0 with value: 46.66515400484355.
[I 2021-11-19 16:57:56,085] Trial 1 finished with value: 0.596846951034281 and parameters: {'x': 2.772558703940536}. Best is trial 1 with value: 0.596846951034281.
